In [1]:
import numpy as np

In [2]:
class Job:
    def __init__(self,job_no,machine_no,process_time,n_tasks):
        
        self.job_no=job_no
        self.process_time=process_time
        self.machine_no=machine_no
        self.cur_process_time=self.process_time[self.machine_no-1]
        self.n_tasks=n_tasks    
        self.job_status=0             #0-Available , 1-In queue ,2-In progress
        self.machines_done=[]         #machines it has undergone working in
        
    def update_pt(self):
        #print("updated process tie of job ",self.job_no," pt ",)
        self.process_time[self.machine_no-1]=self.cur_process_time

In [3]:
class Machine:
    def __init__(self,machine_no,job_list):
        
        self.machine_no=machine_no
        self.job_queue=[]
        self.cur_job=None
        for idx,job in enumerate(job_list):
            if job.machine_no==self.machine_no and job.job_status==0:
                self.job_queue.append(job)
                job.job_status=1
                print("added job: ",job.job_no," to machine: ",self.machine_no)
        
    def assign_job(self,job):
        if job!=None:
            print("\njob ",job.job_no," currently worked upon by machine ",self.machine_no," processing time of ",job.cur_process_time)
            self.job_queue.remove(job)
            self.cur_job=job
            self.cur_job.job_status=2
            #adding to the jobs mahcine_done queue so that same job doesn't return back ot the machine
            job.machines_done.append(self)

    def job_finished(self):
        if self.cur_job!=None:
            if self.cur_job.cur_process_time<=0:
                return True
            else:
                return False
        else:
            return True
        
    def job_exit(self):
        temp=self.cur_job
        temp.n_tasks-=1
        temp.job_status=0
        temp.machine_no=-1
        print("\njob ",temp.job_no," exited machine ",self.machine_no)
        self.cur_job=None
        return temp
    
    def add_job(self,job):
        job.job_status=1
        job.machine_no=self.machine_no
        self.job_queue.append(job)        
        #updating the cur process time of the job
        job.cur_process_time=job.process_time[self.machine_no-1]
        print("\njob ",job.job_no," added to machine ",self.machine_no," queue")
        
    def work(self):
        if self.cur_job!=None:
            #print("before",self.cur_job.cur_process_time)
            self.cur_job.cur_process_time-=1
            #print("after",self.cur_job.cur_process_time)
            self.cur_job.update_pt()
        
    def calc_queue_time(self):
        self.total_queue_time=0
        if len(self.job_queue)!=0:
            for job in self.job_queue:
                self.total_queue_time+=job.cur_process_time
            if self.cur_job!=None:
                self.total_queue_time+=self.cur_job.cur_process_time

In [4]:
class Environment:
    def __init__(self,**kwargs):
        self.clock=0
        self.job_list=[Job(1,1,[1,2,3],3),Job(2,1,[2,3,4],3),Job(3,1,[3,4,5],3),Job(4,2,[4,5,6],3),Job(5,3,[5,6,7],3),Job(6,3,[6,7,8],3)]
        self.total_jobs=len(self.job_list)
        self.job_priority_order=[[1,2,0.33,0.5,0.5,0.25],
                                 [0.5,1,0.25,0.33,0.33,0.2],
                                 [3,4,1,2,2,0.5],
                                 [2,3,0.5,1,1,0.33],
                                 [2,3,0.5,1,1,0.33],
                                 [4,5,2,3,3,1]]
        self.job_pool=[]
        
        self.machines=[]
        
    def addMachine(self,no_of_machine):
        for m in range(1,no_of_machine+1):
            machine=Machine(m,self.job_list)
            self.machines.append(machine)
        print("done adding machines")
        
    def find_machine(self,machine_no):
        for machine in self.machines:
            if machine_no==machine.machine_no:
                return machine
        return None
        
    def spt(self,machine_no):
        machine=self.find_machine(machine_no)
        #print("machine in spt ",machine.machine_no)
        temp_lst=machine.job_queue
        temp_lst.sort(key=lambda j: j.cur_process_time)
        #print(temp_lst)
        #print("\nSPT job ",temp_lst[0].job_no," by machine ",temp_lst[0].machine_no)
        return temp_lst[0]
    
    def mwkr(self,machine_no):
        machine=self.find_machine(machine_no)
        temp_lst=machine.job_queue
        temp_lst.sort(key=lambda j: (sum(j.process_time)-j.cur_process_time))
        #print("\nMWKR job ",temp_lst[-1].job_no," by machine ",temp_lst[-1].machine_no," time remaining ",sum(temp_lst[-1].process_time)-temp_lst[-1].cur_process_time)
        return temp_lst[-1]
    
    def mopnr(self,machine_no):
        machine=self.find_machine(machine_no)
        temp_lst=machine.job_queue
        temp_lst.sort(key=lambda j: j.n_tasks)
        #print("\nMOPNR job ",temp_lst[-1].job_no," by machine ",temp_lst[-1].machine_no," tasks remaining ",temp_lst[-1].n_tasks)
        return temp_lst[-1]
    
    def decide_rule(self,machine_no):
        #check if the machines job queue is empty if yes then send no job selected
        if len(self.find_machine(machine_no).job_queue)==0:
            return None,None
        spt_job=self.spt(machine_no)
        mwkr_job=self.mwkr(machine_no)
        mopnr_job=self.mopnr(machine_no)
        job_array=[spt_job,mwkr_job,mopnr_job]
        AHP_mat=[[0 for i in range(3)] for j in range(3)]
        for i in range(len(job_array)):
            for j in range(len(job_array)):
                row_idx=job_array[i].job_no-1
                col_idx=job_array[j].job_no-1
                AHP_mat[i][j]=self.job_priority_order[row_idx][col_idx]
        #print(AHP_mat)
        AHP_mat=np.array(AHP_mat)
        norm1_vector=np.array([sum(x) for x in zip(*AHP_mat)])
        norm2_mat=np.transpose(np.transpose(AHP_mat)/norm1_vector[:,None])
        norm3_vector=np.sum(norm2_mat,axis=1)
        max_idx=norm3_vector.argmax()
        #rule matrix
        rules=["SPT","MWKR","MOPNR"]
        if rules[max_idx]=="SPT":
            return spt_job,rules[max_idx]
        elif rules[max_idx]=="MWKR":
            return mwkr_job,rules[max_idx]
        elif rules[max_idx]=="MOPNR":
            return mopnr_job,rules[max_idx]
 
    def print_machines_done(self,machines_done):
        for machine in machines_done:
            print("machine ",machine.machine_no,end=" ,")
    
    def start_simulation(self):
        self.clock=0
        while(self.total_jobs!=len(self.job_pool)):
            print("--------------------------------------------------------------------------------")
            print("\nCLOCK ",self.clock)
            
            #checking if all the jobs are finished execution
            finished=True
            if len(self.job_pool)!=0:
                for job in self.job_pool:
                    print("job ",job.job_no," tasks ",job.n_tasks)
                    if job.n_tasks==0:
                        finished=True and finished
                    else:
                        finished=False
                if (self.total_jobs!=len(self.job_pool)):
                    finished=False
            else:
                finished=False
            #To prevent early exit 
            if self.clock!=0:
                
                if finished:
                    print("finished ",finished)
                    break
            
                #chekcing if any machine has finished job or not
                for machine in self.machines:
                    machine.work()
                    if machine.job_finished() and machine.cur_job!=None:
                        new_job=machine.job_exit()
                        self.job_pool.append(new_job)
                        print("job pool updated")

                #Arranging in accending(later we use pop method) the job_pool according to MPR(most processing time remaining)
                self.job_pool.sort(key=lambda job: sum(job.process_time))
                #self.job_pool.reverse()
                
                #printing the job pool
                print("\nJOB POOL::",end=" ")    
                for job in self.job_pool:
                    print("Job",job.job_no,end=" ,")
                print()
                
                #Arrange the machines wrt least processing time
                for machine in self.machines:
                    machine.calc_queue_time()
                self.machines.sort(key=lambda machine: machine.total_queue_time)

                #add jobs to machine queue
                for machine in self.machines:
                    #as long as the queue is not empty or the index doesn't exceed the job_pool size
                    if len(self.job_pool)!=0:
                        if machine in self.job_pool[-1].machines_done:
                            continue
                        #printing the machines done of the job to exit
                        print("job ",self.job_pool[-1].job_no,end=":: ")
                        self.print_machines_done(self.job_pool[-1].machines_done)
                        machine.add_job(self.job_pool.pop())
                    else:
                        break
                
            for machine in self.machines:
                if machine.job_finished():
                    #print("job added to machine ",machine.machine_no," workbench ")
                    j,r=self.decide_rule(machine.machine_no)
                    machine.assign_job(j)
                    print("\nRule used is ",r)
            
            self.clock+=1
        
        print("The entire job queue is finished in ",self.clock," time")

In [5]:
env=Environment()
env.addMachine(3)
env.start_simulation()

added job:  1  to machine:  1
added job:  2  to machine:  1
added job:  3  to machine:  1
added job:  4  to machine:  2
added job:  5  to machine:  3
added job:  6  to machine:  3
done adding machines
--------------------------------------------------------------------------------

CLOCK  0

job  3  currently worked upon by machine  1  processing time of  3

Rule used is  MWKR

job  4  currently worked upon by machine  2  processing time of  5

Rule used is  SPT

job  6  currently worked upon by machine  3  processing time of  8

Rule used is  MWKR
--------------------------------------------------------------------------------

CLOCK  1

JOB POOL:: 
--------------------------------------------------------------------------------

CLOCK  2

JOB POOL:: 
--------------------------------------------------------------------------------

CLOCK  3

job  3  exited machine  1
job pool updated

JOB POOL:: Job 3 ,
job  3:: machine  1 ,
job  3  added to machine  2  queue

job  1  currently worked